# Energy efficient parallel programming

**Note:** this notebook is meant to be executed on the supplied Docker image.

TO DO - EN

Celem zajęć jest zaznajomienie się z technikami programowania wydajnego energetycznie oraz porównanie czasu wykonania z ilością zużytej energii.

In [1]:
import os
from pathlib import Path
import time

from multiprocessing import Pool
from collections import defaultdict, deque
import random


## RAPL sysfs Interface


First of all, it's important to find the zone of your CPU in sysfs. Use the knowledge from the previous modules to find the path of thiz zone and complete the code below.

Please verify your path to the RAPL module in your PC.


In [2]:
cpu_zone = '/sys/devices/virtual/powercap/intel-rapl/intel-rapl:0/'

Let's verify whether this zone seems like the zone of your CPU.

In [3]:
if os.path.isdir(cpu_zone):
    print('✓ Zone exists')
else:
    print('✗ Zone does not exist!')

name = Path(f'{cpu_zone}/name').read_text().strip()

if name.startswith('package-'):
    print('✓ Its name starts with \'package-\'')
else:
    print('✗ Its zone does not start with \'package-\'!')

if os.path.isfile(f'{cpu_zone}/energy_uj'):
    print('✓ File \'energy_uj\' exists')
else:
    print('✗ File \'energy_uj\' does not exist!')

✓ Zone exists
✓ Its name starts with 'package-'
✓ File 'energy_uj' exists


The energy counter which reports energy consumed by the zone in micro joules is available as a file named `energy_uj`. This file returns the energy consumed from ar arbitrary, but fixed point in time. By calculating the difference between these values at different points in time, we can obtain the energy consumed during the measurrement period.


Let's start with rading the value of `energy_uj`. Complete the following code.

In [4]:
def energy_uj():
    # TO DO - remove implementation
    fp = open(cpu_zone+'energy_uj', 'r')
    return int(fp.read())

print(f'Current energy counter value: {energy_uj()}')

Current energy counter value: 43587240406


Next, we can use the energy counter value read at two different times to calculate energy consumption during the measurement period. Complete the following code. Remembebr that the counter may be reset to zero!


In [5]:
def energy_consumption(energy_uj_start, energy_uj_end):
    # TO DO - remove implementation
    fp = open(cpu_zone+'max_energy_range_uj', 'r')
    max_energy_range_uj = int(fp.read())

    if energy_uj_end < energy_uj_start:
        return max_energy_range_uj-energy_uj_start+energy_uj_end
    return energy_uj_end-energy_uj_start

Let's verify whether the function defined above works. Check the output of the script below.

In [6]:
energy = energy_consumption(10000, 10000)
if energy == 0:
    print('✓ Pass')
else:
    print(f'✗ Fail: {energy}')

energy = energy_consumption(0, 10000)
if energy == 10000:
    print('✓ Pass')
else:
    print(f'✗ Fail: {energy}')

counter_max = int(Path(f'{cpu_zone}/max_energy_range_uj').read_text().strip())
energy = energy_consumption(counter_max - 1000, 1000)
if energy == 2000:
    print('✓ Pass')
else:
    print(f'✗ Fail: {energy}')

✓ Pass
✓ Pass
✓ Pass


Let's try measuring idle CPU energy consumption using the functions `energy_uj()` and `energy_consumption()`. Complete the following code to get the energy consumed during the 5 second measurement period in joules.

In [7]:
measure_time = 5  # seconds

start_energy_uj = energy_uj()
time.sleep(measure_time)
end_energy_uj = energy_uj()

energy_consumed_uj = energy_consumption(start_energy_uj, end_energy_uj)
energy_consumed_j = energy_consumed_uj / 10**6 / measure_time

print(f'Energy consumed: {energy_consumed_j} J')

Energy consumed: 37.440352000000004 J


The energy consumed should be roughly between 10 and 1000 joules depending on the power of your equipment.

**Note**: If you use the ultrabook with low energy consumprion CPU like [Intel® Core™ i7-1165G7](https://ark.intel.com/content/www/us/en/ark/products/208921/intel-core-i7-1165g7-processor-12m-cache-up-to-4-70-ghz-with-ipu.html) the consumed energy may be below 10 joules

Verify the usage after power capping

In [10]:
class Graph:
    def __init__(self):
        self.graph = defaultdict(list)

    def add_edge(self, u, v):
        self.graph[u].append(v)

    def bfs(self, start):
        visited = set()
        queue = deque([start])
        result = []

        while queue:
            node = queue.popleft()
            if node not in visited:
                visited.add(node)
                result.append(node)
                queue.extend(self.graph[node])

        return result

    def dfs(self, start):
        visited = set()
        stack = [start]
        result = []

        while stack:
            node = stack.pop()
            if node not in visited:
                visited.add(node)
                result.append(node)
                stack.extend(neighbor for neighbor in reversed(self.graph[node]) if neighbor not in visited)

        return result

def generate_random_graph(num_nodes, num_edges) -> Graph:
    graph = Graph()
    for _ in range(num_edges):
        u = random.randint(1, num_nodes)
        v = random.randint(1, num_nodes)
        graph.add_edge(u, v)
    return graph

def bfs_wrapper(graph: Graph, start_node):
    return graph.bfs(start_node)

def dfs_wrapper(graph: Graph, start_node):
    return graph.dfs(start_node)


In [11]:
# Example usage:
num_nodes = 10
num_edges = 20
graph = generate_random_graph(num_nodes, num_edges)

# Choose the number of processes
num_processes = 2

# Create a pool of processes
pool = Pool(processes=num_processes)

# Run BFS and DFS in parallel
start_node = 1
bfs_result = pool.apply_async(bfs_wrapper, (graph, start_node,))
dfs_result = pool.apply_async(dfs_wrapper, (graph, start_node,))

# Get the results
bfs_result = bfs_result.get()
dfs_result = dfs_result.get()

print("BFS Result:", bfs_result)
print("DFS Result:", dfs_result)

# Close the pool
pool.close()
pool.join()

BFS Result: [1, 8, 9, 2, 4, 3, 5, 10, 7]
DFS Result: [1, 8, 9, 4, 2, 3, 10, 7, 5]
